In [1]:
#%%
import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from PIL import Image


Pickle

Deserializing dictionary saved in pickle

In [4]:
Exp_2_pckl={}
path = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2/Arttu/'
with open(path + 'Exp_design_2_ID_1.pkl', 'rb') as f:
    Exp_2_pckl = pickle.load(f)

Keys of nested dictionaries; Keys-indexes nested in the first internal dictionary


In [5]:
print('Length of the first mask:', len(Exp_2_pckl[0]["masks"]))
print('Number of masks',len(Exp_2_pckl.keys()))
print('Variables for each mask:',Exp_2_pckl[0].keys())

Length of the first mask: 53
Number of masks 287
Variables for each mask: dict_keys(['masks', 'x', 'y', 'z', 'time', 'area'])


CSV

In [35]:
Exp_2_csv = pd.read_csv(path +'/Exp_design_2_1.csv', index_col=None) #loading the Arttu's csv file to match the measurement_id and well_id, to be matched with the pckl info

print('Variables for each row-mask:',Exp_2_csv.columns)
#print('Size of csv file:' )
print('Number of masks:',Exp_2_csv.shape[0])
print('Number of unique IDs in csv file:',len(Exp_2_csv["ID_running"].unique()))
print('Number of groups after measurement and well_id grouping:',(Exp_2_csv.groupby(['measurement_id', 'well_id'])).ngroups)


print(Exp_2_csv.loc[28,'day'])

Exp_2_csv.loc[Exp_2_csv['ID_running'] == key, 'day'].iloc[0]


Variables for each row-mask: Index(['Unnamed: 0', 'seeding_density', 'cell_label', 'well_id',
       'measurement_id', 'matrix', 'ID', 'day', 'ID_running', 'location'],
      dtype='object')
Number of masks: 287
Number of unique IDs in csv file: 287
Number of groups after measurement and well_id grouping: 178
240520


Metadata from nd2 files

In [27]:
Exp_2_json = pd.read_json('C:/Users/srboval1/InstruProject/NikonPipes/dataStore/metalib.json') 
print(Exp_2_json.get(key="240521").get("m"))

def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    return pixel_transf

print(search_json(240521))


0.660045336571487
0.660045336571487


In [12]:

concentration_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
material_mapping = {
'2mgml': '1',
'3mM': '2',
'40mM': '3'
}
reverse_material_mapping = {
'2mgml':'collagen 2mg/ml',
'3mM':'IPN 2.5mM',
'40mM':'IPN 40mM'
}
cell_mapping = {
'MCF10A': '1',
'DCIS.COM': '2',
'MCF10A.DCIS.COM': '2',
'MCF10AT': '3'
}


In [11]:
pixel_size = 0.32*1e-6

print('List of contour coordinates:',Exp_2_pckl[2]["masks"][10])
print('Linda calculated area [pixels]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10]))
print('re-calculated area in [m]:',cv2.contourArea(Exp_2_pckl[2]["masks"][10])*(0.660535171355723**2))

print('Arttu calculated area:',Exp_2_pckl[2]["area"][10])

List of contour coordinates: [[[1298  647]]

 [[1297  648]]

 [[1296  648]]

 ...

 [[1301  648]]

 [[1300  648]]

 [[1299  647]]]
Linda calculated area [pixels]: 69341.0
re-calculated area in [m]: 30253.94375825337
Arttu calculated area: 30253.94375825337


In [43]:
#%%
#pixel_size = 0.32*1e-6
df_collecting = []
df_all = []

def search_csv(key_id):
    well = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'well_id'].iloc[0]
    measurement = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'measurement_id'].iloc[0]
    cell_line = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'cell_label'].iloc[0]
    material = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'matrix'].iloc[0]
    day = Exp_2_csv.loc[Exp_2_csv['ID_running'] == key_id, 'day'].iloc[0]
    return well, measurement, cell_line, material, day


def search_json(parameter):
    pixel_transf = Exp_2_json.get(key=str(parameter)).get("m")
    return pixel_transf

for key in Exp_2_pckl.keys() :
    df_temp =[] 

    for i in range(len(Exp_2_pckl[key]["time"])): #looping through time
        mask = Exp_2_pckl[key]["masks"][i]
        time = Exp_2_pckl[key]["time"][i]
        #area_Arttu = Exp_2_pckl[key]["area"][i]    
        
        well_id,measurement_id, cell, matrix, date = search_csv(key)

        area_pixels2 = cv2.contourArea(mask)        
        area_µm = area_pixels2*(search_json(date)**2)    

        perimeter = cv2.arcLength(mask, closed=True)*search_json(date) #perimeter for each contour
        compactness = 4 * math.pi * area_µm  #compactness (spheroid area/area of object with the identical perimeter)
        rect = cv2.minAreaRect(mask) #minimal rectangle
        width, height  = rect[1] #extracting the tuple containing rectangular dimensions
        aspect_ratio = width / height
        convexhull = cv2.convexHull(mask)
        convexhull_perimeter = cv2.arcLength(convexhull, closed=True)*search_json(date) #perimeter for each contour
        convexhull_area = cv2.contourArea(convexhull)*(search_json(date)**2)
        convexity_perimeter = convexhull_perimeter/perimeter
        convexity_area = convexhull_area/area_µm
        
        info={}
        info = {
            "spheroid_id": key,
            "time_id": i+1,
            "area_pixels**2":area_pixels2,
            "area_µm": area_µm,
            "perimeter_µm": perimeter,
            "compactness": compactness,
            "aspect_ratio": aspect_ratio,
            "convex hull perimeter": convexhull_perimeter,
            "convex hull area": convexhull_area,
            "convexity_perimeter": convexity_perimeter,
            "convexity_area": convexity_area,
            "well_id":well_id,
            "measurement_id":measurement_id,
            "cell_line":cell,
            "matrix":matrix,
            "date":date
            }
        df_temp.append(pd.DataFrame([info]) )  #appending the info to the contours_info, which collects all timestemps
    
    df_collecting.append(pd.concat(df_temp,ignore_index=True)) #lists of dfs are contantenated and appended to df_collecting, thedf_temp is re-initialized

df_all=pd.concat(df_collecting, ignore_index=True)
df_all['running_value'] = df_all.groupby(['measurement_id', 'well_id']).ngroup()

saving_path= 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/Exp2'
df_all.to_csv(saving_path+'/contours_exp2.csv')  

Visualizing the mask

In [38]:
#%%
save = 'C:/Users/srboval1/OneDrive - Aalto University/Instru/Datafiles/TimeLapse/'
width, height = 2304, 2304

# Create a blank white image using Pillow and converting it into array


for key in Exp_2_pckl.keys() :
    time = Exp_2_pckl[key]["time"]
    
    blank_image = Image.new('RGB', (width, height), color = (255, 255, 255))
    blank_image_array = np.asarray(blank_image)

    new_image=np.zeros((height, width, 3), np.uint8)


    for i in range(len(Exp_2_pckl[key]["time"])): #looping through time
        mask = Exp_2_pckl[key]["masks"][i]
        spheroid_id = key+1
        blank_image_array= cv2.drawContours(blank_image_array, Exp_2_pckl[key]["masks"][i], -1, (0, 0, 0), 1)
        #print( f'{unique_material}- {unique_line}; min: {min_value} and max {max_value}')

    plt.title(f'{matrix} - {cell_line}; spheroid id: {spheroid_id}') 
    plt.imshow(new_image)
    filename = f'{cell_line}_{matrix}_id_{id}.tiff'

    cv2.imwrite(save+filename, blank_image_array) #saving the contours

NameError: name 'cell_line' is not defined